In [1]:
# pip installs - ignore the error message!

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3    datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib
!pip install --upgrade transformers==4.49.0 accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# -----------------------------
# Imports
# -----------------------------
import os
from datetime import datetime
from tqdm import tqdm
from google.colab import userdata

import torch
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from datasets import load_dataset
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
import matplotlib.pyplot as plt


In [7]:
# -----------------------------
# Constants
# -----------------------------
BASE_MODEL = "tiiuae/falcon-7b-instruct"   # Falcon instead of LLaMA
PROJECT_NAME = "pricer"
HF_USER = "fakharbutt44"               # Replace with your HF username
DATASET_NAME = "ed-donner/pricer-data"    # Your fine-tuning dataset

MAX_SEQUENCE_LENGTH = 182
RUN_NAME = f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

In [8]:
# LoRA Hyperparameters
LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"]  # Falcon-specific modules
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

# Training Hyperparameters
EPOCHS = 1
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

%matplotlib inline


In [9]:
# -----------------------------
# Login to Hugging Face + WandB
# -----------------------------
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

In [10]:
# -----------------------------
# Load Dataset
# -----------------------------
dataset = load_dataset(DATASET_NAME)
train = dataset["train"]
test = dataset["test"]

# Reduce dataset to 50,000 random samples for faster training
train = train.shuffle(seed=42).select(range(min(500, len(train))))

if LOG_TO_WANDB:
    wandb.init(project=PROJECT_NAME, name=RUN_NAME)

README.md:   0%|          | 0.00/416 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/914k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
# -----------------------------
# Quantization Config
# -----------------------------
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )
else:
    quant_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16
    )

In [12]:
# -----------------------------
# Load Tokenizer & Model
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

base_model.generation_config.pad_token_id = tokenizer.pad_token_id
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration_falcon.py: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Memory footprint: 3904.5 MB


In [13]:
# -----------------------------
# Data Collator
# -----------------------------
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [14]:
# -----------------------------
# LoRA Config
# -----------------------------
lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES
)

# -----------------------------
# Training Config
# -----------------------------
train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)


In [17]:
# -----------------------------
# Fine-tuning
# -----------------------------
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
)

fine_tuning.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:139: UserWarning: Could not find response key `Price is $` in the following instance: How much does this cost to the nearest dollar?

Polaris Off Road Sound Bar 4 by MB Quart®
Don't let its smaller size fool you ‐ the Sound Bar 4 can jam as hard as any other sound bar, maybe more. This Accessory is Designed to Fit the Following 2019 RZR 900 --, 2019 RZR 900 EPS, 2019 RZR S 1000 EPS, 2019 RZR S4 1000 EPS, 2019 RZR S 900 --, 2019 RZR S 900 EPS, 2019 RZR XP 1000 EPS, 2019 RZR XP 1000 DYNAMIX Edition EPS, 2019 RZR XP 1000 High Lifter Edition EPS, 2019 R. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:139: UserWarning: Could not find response key `Price is $` in the following instance: How much does this cost to the nearest dollar?

Rasalas 16GB Kit DDR3 Ram 16GB 2Rx8 PC3 10600U D

Step,Training Loss
50,3.606000
100,2.863500


/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:139: UserWarning: Could not find response key `Price is $` in the following instance: How much does this cost to the nearest dollar?

IMALENT SR16 Spotlight Flashlight 55000 Lumens Super Bright Led Rechargeable Flashlights, Cree HI LEDs Waterproof Rechargeable Spotlight Long Beam 1715 Meters for Searching and Emergency
SUPER BRIGHT FLASHLIGHT IMALENT SR16 Led flashlight high lumen is equipped with an upgraded HI leds. This LED flashlight can produce super high output up to 55,000 lumens, which is 10 times brighter than regular flashlights. It can easily illuminate a few football field or focus on objects up to 1715 meters away. Put the flashlight accessories in the gift box, which is very convenient to carry. RECHARGEABLE FLASHLIGHT & USB CHARGING | WITH HANDLE The rechargeable flashlight has been completely upgraded to shorten the charging time and extend the

Price is . This instance will be ignored in loss calculation. Not

KeyboardInterrupt: 

In [16]:


# -----------------------------
# Push to Hugging Face Hub
# -----------------------------
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

if LOG_TO_WANDB:
    wandb.finish()

README.md:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ..._07.12.59/adapter_model.safetensors:  13%|#2        | 33.5MB /  261MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Saved to the hub: pricer-2025-08-05_07.12.59


train/epoch,▁▆█
train/global_step,▁▆█
train/grad_norm,▁█
train/learning_rate,█▁
train/loss,█▁
total_flos,3646988464088064.0
train/epoch,1
train/global_step,125
train/grad_norm,2.72769
train/learning_rate,1e-05
train/loss,2.861
